In [75]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from rbf_neural_networks import RBFNeuralNetwork
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay

In [76]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [77]:
rbf_nn = RBFNeuralNetwork(num_centers=10)
rbf_nn.fit(X_train, y_train)
y_pred = rbf_nn.predict(X_test)

weights: [[ 8.41567993e-04  1.88497110e-01 -2.82733459e-02]
 [ 3.48178327e-01  1.01931395e-02  2.32080548e-04]
 [-9.17914413e-03  1.34373100e-01  8.94828283e-02]
 [-1.83870265e-02  8.78383229e-03  2.21566369e-01]
 [ 2.03779712e-02  2.12562955e-01  6.69433825e-03]
 [-2.06763366e-02  8.02394676e-02  1.39285971e-01]
 [ 3.68915537e-01 -1.94520805e-02 -4.44234929e-03]
 [ 1.23333372e-02 -2.61221812e-02  2.10317707e-01]
 [-8.48291126e-03  1.11310468e-01  1.29570044e-01]
 [ 5.23672281e-02  1.45685290e-01 -4.41104716e-02]]


In [78]:
accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

classes = np.unique(y_test)
for c in classes:
    classes_acc = np.mean(y_pred[y_test == c] == y_test[y_test == c])
    print(f"Accuracy class {c}: {classes_acc*100:.2f}%")

Test Accuracy: 96.67%
Accuracy class 0: 100.00%
Accuracy class 1: 100.00%
Accuracy class 2: 90.91%


In [79]:
print(" " , rbf_nn.hihi)

AttributeError: 'RBFNeuralNetwork' object has no attribute 'hihi'